# Data Setup

In [ ]:
import torch
import torch.nn as nn
from torchinfo import summary

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [22]:
if torch.cuda.is_available():
    print("GPU available.")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
    device = torch.device("cuda")
elif torch.mps.is_available():
    print("MPS available.")
    print(f"Using MPS: {torch.mps.device_count()}")
    device = torch.device("mps")
else:
    print("Using CPU.")
    device = torch.device("cpu")

device = torch.device("cpu")

torch.set_default_device(device=device)

MPS available.
Using MPS: 1


In [23]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head(5)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [24]:
df.drop(columns = ['id', 'Unnamed: 32'], inplace = True)

In [25]:
x_train, x_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

In [26]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [27]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [28]:
x_train_tensor = torch.from_numpy(x_train).to(dtype=torch.float32, device=device)
x_test_tensor = torch.from_numpy(x_test).to(dtype=torch.float32, device=device)
y_train_tensor = torch.from_numpy(y_train).to(dtype=torch.float32, device=device)
y_test_tensor = torch.from_numpy(y_test).to(dtype=torch.float32, device=device)

# Defining Model

In [29]:
class NeuralNetwork(nn.Module):

    def __init__(self, num_features):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(in_features=num_features, out_features=3, device=device),
            nn.ReLU(),
            nn.Linear(in_features=3, out_features=1, device=device),
            nn.Sigmoid(),
        )
    
    def forward(self, features):
        out = self.network(features)
        return out

# Parameters

In [30]:
lr = 0.1
epochs = 1000
batch_size = 32
n_samples = len(x_train_tensor)

In [31]:
loss_function = nn.BCELoss()

# Training Pipeline

In [32]:
# Create Model
model = NeuralNetwork(x_train_tensor.shape[1]).to(device=device)

x_train_tensor = x_train_tensor.to(device)
y_train_tensor = y_train_tensor.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=lr)

# Define loop
for epoch in range(epochs):
        
    for start_idx in range (0, n_samples, batch_size):
        end_idx = start_idx + batch_size
        x_batch = x_train_tensor[start_idx:end_idx]
        y_batch = y_train_tensor[start_idx:end_idx]

        # Forward Pass
        y_pred = model(x_batch)

        # Loss Calculation
        loss = loss_function(y_pred, y_batch.view(-1, 1))
        print(f"Epoch: {epoch + 1}, Loss: {loss.item()}")
    
        # Zero Gradients
        optimizer.zero_grad()

        # Backward Pass
        loss.backward()

        # Paramers Update
        optimizer.step()
    

Epoch: 1, Loss: 0.6414557099342346
Epoch: 1, Loss: 0.6268121004104614
Epoch: 1, Loss: 0.7323487401008606
Epoch: 1, Loss: 0.6693878173828125
Epoch: 1, Loss: 0.6260678172111511
Epoch: 1, Loss: 0.6298712491989136
Epoch: 1, Loss: 0.6762980222702026
Epoch: 1, Loss: 0.6258065700531006
Epoch: 1, Loss: 0.6589031219482422
Epoch: 1, Loss: 0.6177828907966614
Epoch: 1, Loss: 0.6157267689704895
Epoch: 1, Loss: 0.7204750776290894
Epoch: 1, Loss: 0.6496188640594482
Epoch: 1, Loss: 0.5971440076828003
Epoch: 1, Loss: 0.5683420896530151
Epoch: 2, Loss: 0.5719625949859619
Epoch: 2, Loss: 0.5357909798622131
Epoch: 2, Loss: 0.6532560586929321
Epoch: 2, Loss: 0.6060678958892822
Epoch: 2, Loss: 0.5402870178222656
Epoch: 2, Loss: 0.5440483689308167
Epoch: 2, Loss: 0.6040878295898438
Epoch: 2, Loss: 0.555616557598114
Epoch: 2, Loss: 0.597427487373352
Epoch: 2, Loss: 0.5312601327896118
Epoch: 2, Loss: 0.5359203219413757
Epoch: 2, Loss: 0.6590016484260559
Epoch: 2, Loss: 0.5693547129631042
Epoch: 2, Loss: 0.5025

In [33]:
summary(model)

Layer (type:depth-idx)                   Param #
NeuralNetwork                            --
├─Sequential: 1-1                        --
│    └─Linear: 2-1                       93
│    └─ReLU: 2-2                         --
│    └─Linear: 2-3                       4
│    └─Sigmoid: 2-4                      --
Total params: 97
Trainable params: 97
Non-trainable params: 0

# Evaluation

In [34]:
with torch.no_grad():
    y_pred = model(x_test_tensor)
    y_pred = (y_pred > 0.95).float()
    accuracy = (y_pred == y_test_tensor).float().mean()
    print(f"Accuracy: {accuracy.item()}")

Accuracy: 0.5637118816375732
